In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle 

In [2]:
# Load the dataset
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
## encode categorical variable
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

In [7]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [9]:
## One hot encode geography column
from sklearn.preprocessing import OneHotEncoder
geo = OneHotEncoder()
df_geo = geo.fit_transform(df[['Geography']])
df_geo

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [12]:
df_geo.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [10]:
geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [15]:
df_en = pd.DataFrame(df_geo.toarray(), columns=geo.get_feature_names_out(['Geography']))
df_en

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [16]:
## combine all the columns with the original data
df = df.drop(['Geography'], axis=1)
df = pd.concat([df, df_en], axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [17]:
## save the encoders and scaler 
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(label_encoder_gender, f)

with open('geo_encoder.pkl','wb') as f:
    pickle.dump(geo, f)

In [18]:
## divide the data into independent and dependent variables
X = df.drop(['Exited'], axis=1)
y = df['Exited']

# split the data in training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [20]:
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler, f)

# ANN IMplemntation

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [24]:
(X_train.shape[1],)

(12,)

In [27]:
## build our ANN model

model = Sequential([
    Dense(64, activation = 'relu', input_shape=(X_train.shape[1],)), ## first hiddent layer connected with input layer
    Dense(32, activation = 'relu'), ## HL2
    Dense(1, activation = 'sigmoid') ## output layer
])

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
los = tensorflow.keras.losses.BinaryCrossentropy(from_logits=False)

In [33]:
## compile the model
## Here in optimizer Adam has its own build in learning rate,
### but we can also set the learning rate with the code above 
model.compile(optimizer=opt, loss=los, metrics=['accuracy'])

In [ ]:
## set up the tensorboard
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

log_dir = 'logs/fit' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [42]:
## set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [43]:
## Training the model
history = model.fit(
    X_train, y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks= [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3383 - accuracy: 0.8624 - val_loss: 0.3503 - val_accuracy: 0.8525
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3394 - accuracy: 0.8630 - val_loss: 0.3507 - val_accuracy: 0.8600
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3365 - accuracy: 0.8630 - val_loss: 0.3378 - val_accuracy: 0.8650
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3330 - accuracy: 0.8616 - val_loss: 0.3586 - val_accuracy: 0.8590
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3311 - accuracy: 0.8639 - val_loss: 0.3536 - val_accuracy: 0.8590
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3296 - accuracy: 0.8669 - val_loss: 0.3459 - val_accuracy: 0.8605
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3257 - accuracy: 0.8685 - val_loss: 0.3458 - val_accuracy: 0.8600

In [44]:
model.save('model.h5')

c:\Users\divya\python\ANN\venv1\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [50]:
## Load tensorboard Extention
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [54]:
%tensorboard --logdir logs/fit20250506-125301

Reusing TensorBoard on port 6007 (pid 27076), started 0:09:01 ago. (Use '!kill 27076' to kill it.)

In [ ]:
## Load the pickle file
